In [1]:
import utils

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, Normalization

2024-09-10 19:29:57.999663: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-10 19:29:58.000052: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-10 19:29:58.002333: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-10 19:29:58.009413: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-10 19:29:58.021741: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [2]:
input_features = ['year',
                  'matches_won_percent',
                  'podium_score_yearly',
                  'matches_won_percent_2',
                  'podium_score_yearly_2',]
output_feature = 'winner'

In [3]:
matches = utils.get_matches(with_team_stats=True, duplicate_with_reversed=True)
matches

,id,year,team1,score1,score2,team2,score_diff,winner,matches_played,matches_won,...,podium_score_yearly,cups_won_yearly,matches_played_2,matches_won_2,years_played_2,podium_score_2,cups_won_2,matches_won_percent_2,podium_score_yearly_2,cups_won_yearly_2
0,0,1950,Brazil,4,0,Mexico,4,0,77.0,65.0,...,6.000000,0.294118,35.0,14.0,14.0,0.0,0.0,40.000000,0.000000,0.0
1,1,1950,Yugoslavia,3,0,Switzerland,3,0,26.0,14.0,...,0.250000,0.000000,22.0,9.0,8.0,0.0,0.0,40.909091,0.000000,0.0
3,3,1950,Yugoslavia,4,1,Mexico,3,0,26.0,14.0,...,0.250000,0.000000,35.0,14.0,14.0,0.0,0.0,40.000000,0.000000,0.0
4,4,1950,Brazil,2,0,Yugoslavia,2,0,77.0,65.0,...,6.000000,0.294118,26.0,14.0,8.0,2.0,0.0,53.846154,0.250000,0.0
5,5,1950,Switzerland,2,1,Mexico,1,0,22.0,9.0,...,0.000000,0.000000,35.0,14.0,14.0,0.0,0.0,40.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1528,761,2014,Argentina,3,2,Nigeria,1,0,54.0,36.0,...,2.857143,0.142857,14.0,5.0,5.0,0.0,0.0,35.714286,0.000000,0.0
1529,762,2014,Iran,1,3,Bosnia Herzegovina,-2,1,9.0,1.0,...,0.000000,0.000000,3.0,1.0,1.0,0.0,0.0,33.333333,0.000000,0.0
1530,763,2014,Germany,1,0,United States,1,0,80.0,61.0,...,5.875000,0.187500,22.0,6.0,8.0,0.0,0.0,27.272727,0.000000,0.0
1531,764,2014,Ghana,1,2,Portugal,-1,1,9.0,4.0,...,0.000000,0.000000,22.0,13.0,6.0,6.0,0.0,59.090909,1.000000,0.0


In [4]:
train, test = train_test_split(matches, test_size=0.2)

train_inputs = train[input_features].values
test_inputs = test[input_features].values

train_outputs = train[output_feature].values
test_outputs = test[output_feature].values

In [5]:
network = Sequential([
    Input((len(input_features),)),
    Normalization(),
    Dense(10, activation='sigmoid'),
    Dense(10, activation='sigmoid'),
    Dense(1, activation='sigmoid'),
])

network.layers[0].adapt(train_inputs)

network.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy',],
)


network.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 5)              │            11 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 10)             │            60 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           110 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 192 (772.00 B)

 Trainable params: 181 (724.00 B)

 Non-trainable params: 11 (48.00 B)

In [6]:
history = network.fit(
    train[input_features], train[output_feature],
    epochs=50,
    batch_size=128,
)

Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.4681 - loss: 0.6993  
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 910us/step - accuracy: 0.4657 - loss: 0.6943
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 835us/step - accuracy: 0.4983 - loss: 0.6881
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 858us/step - accuracy: 0.5703 - loss: 0.6855
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 892us/step - accuracy: 0.7070 - loss: 0.6839
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 847us/step - accuracy: 0.7258 - loss: 0.6815
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 878us/step - accuracy: 0.7006 - loss: 0.6807
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 857us/step - accuracy: 0.7098 - loss: 0.6783
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7156 - loss: 0.6757 
Epoch 10/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - accuracy: 0.7035 - loss: 0.6741
Epoch 11/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 793us/step - accuracy: 0.7230 - loss: 0.6718
Epoch 12/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 783us/step - accuracy: 0.7349 - los

In [7]:
test_predictions = network.predict(test[input_features])

accuracy_score(test[output_feature], test_predictions.round()) 

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


0.8050847457627118

In [8]:
def predict(year, team1, team2):
    case_inputs = utils.build_inputs_for_match(year, team1, team2, input_features)
    result = network.predict(case_inputs)
    
    if result < 0.5:
        winner = team1
    else:
        winner = team2
        
    print(result[0][0], '→', winner)

In [9]:
predict(1950, 'Mexico', 'Brazil')  # real result: 4-0 wins Brazil

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
0.7971719 → Brazil


In [10]:
predict(1990, 'United Arab Emirates', 'Colombia')  # real result: 2-0 wins Colombia

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
0.71419024 → Colombia


In [11]:
predict(2002, 'South Africa', 'Spain')  # real result: 2-3 wins Spain

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
0.7114432 → Spain


In [12]:
predict(2010, 'Japan', 'Cameroon')  # real result: 1-0 wins Japan

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
0.49351278 → Japan


In [13]:
predict(2014, 'Argentina', 'Germany')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
0.6070076 → Germany
